In [ ]:
pip install transformers datasets seqeval torch evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=196ea30bb4f8e24a1ae5f9114d9cb31b5d4d4d7a632c8a17697517a7e671c438
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset
from evaluate import load
import torch


In [ ]:
from google.colab import files

# This will prompt you to upload the file
uploaded = files.upload()


Saving labeled_telegram_subset.conll to labeled_telegram_subset.conll


In [ ]:
import pandas as pd

# Get the filename from the uploaded dictionary
filename = list(uploaded.keys())[0]
print(f'Loading file: {filename}')

# Initialize lists to hold tokens and labels
tokens = []
labels = []

# Read the file line by line
with open(filename, 'r') as file:
    for line in file:
        line = line.strip()
        if line:  # Skip empty lines
            parts = line.split(maxsplit=1)  # Split by the first space only
            if len(parts) == 2:  # Ensure there are exactly 2 parts
                token, label = parts
                tokens.append(token)
                labels.append(label)
            else:
                print(f"Skipping line: {line}")  # Debugging output for lines that don't match the format

# Create a DataFrame
df = pd.DataFrame({'Token': tokens, 'Label': labels})

# Display the first few rows of the DataFrame to ensure it was loaded correctly
print(df.head(10))


Loading file: labeled_telegram_subset.conll
                                 Token      Label
0  ...................................          O
1                             Electric          O
2                                 Hair          O
3                              Steamer          O
4                                  Cap          O
5                                 High          O
6                              Quality          O
7                                 የፀጉር          O
8                                እስቲመር  B-PRODUCT
9                               ትሪትመንት          O


In [ ]:
# Create a label mapping
label_mapping = {
    "O": 0,
    "I-LOC": 1,
    "I-PRICE": 2,
    "I-Product": 3,
    # Add other labels as needed
}

# Map the labels in the DataFrame
df['Label'] = df['Label'].map(label_mapping)

# Drop any rows with NaN values after mapping
df.dropna(inplace=True)

# Convert labels to integer type
df['Label'] = df['Label'].astype(int)

# Display the DataFrame after mapping
print(df.head(10))


                                  Token  Label
0   ...................................      0
1                              Electric      0
2                                  Hair      0
3                               Steamer      0
4                                   Cap      0
5                                  High      0
6                               Quality      0
7                                  የፀጉር      0
9                                ትሪትመንት      0
10                                  ወይም      0


In [ ]:
from transformers import XLMRobertaTokenizerFast, XLMRobertaForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset
from evaluate import load
import torch

# Load the fast tokenizer and model for XLM-Roberta
xlm_roberta_tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
xlm_roberta_model = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(label_mapping))

def tokenize_and_align_labels_xlm_roberta(examples):
    tokenized_inputs = xlm_roberta_tokenizer(
        examples['Token'].tolist(),
        padding=True,
        truncation=True,
        is_split_into_words=True,
    )

    labels = []
    word_ids = tokenized_inputs.word_ids(batch_index=0)  # Assuming batch_size is 1
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            label_ids.append(examples['Label'].values[word_idx])
        else:
            label_ids.append(-100)
        previous_word_idx = word_idx
    tokenized_inputs["labels"] = label_ids
    return tokenized_inputs

# Apply the function to each sentence
tokenized_datasets_xlm_roberta = [tokenize_and_align_labels_xlm_roberta(sentence) for _, sentence in df.groupby(df.index // 10)]

# Convert lists to PyTorch tensors
input_ids_xlm_roberta = [torch.tensor(item['input_ids'], dtype=torch.long) for item in tokenized_datasets_xlm_roberta]
attention_mask_xlm_roberta = [torch.tensor(item['attention_mask'], dtype=torch.long) for item in tokenized_datasets_xlm_roberta]
labels_xlm_roberta = [torch.tensor(item['labels'], dtype=torch.long) for item in tokenized_datasets_xlm_roberta]

# Pad sequences to ensure they have the same length
input_ids_xlm_roberta = torch.nn.utils.rnn.pad_sequence(input_ids_xlm_roberta, batch_first=True, padding_value=xlm_roberta_tokenizer.pad_token_id)
attention_mask_xlm_roberta = torch.nn.utils.rnn.pad_sequence(attention_mask_xlm_roberta, batch_first=True, padding_value=0)
labels_xlm_roberta = torch.nn.utils.rnn.pad_sequence(labels_xlm_roberta, batch_first=True, padding_value=-100)

# Combine into a dataset
dataset_xlm_roberta = Dataset.from_dict({
    'input_ids': input_ids_xlm_roberta,
    'attention_mask': attention_mask_xlm_roberta,
    'labels': labels_xlm_roberta
})

# Set up training arguments for XLM-Roberta
training_args_xlm_roberta = TrainingArguments(
    output_dir='./results_xlm_roberta',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Fine-tune the model
trainer_xlm_roberta = Trainer(
    model=xlm_roberta_model,
    args=training_args_xlm_roberta,
    train_dataset=dataset_xlm_roberta,
    eval_dataset=dataset_xlm_roberta,
)

trainer_xlm_roberta.train()

# Evaluate the model
results_xlm_roberta = trainer_xlm_roberta.evaluate()
print(results_xlm_roberta)

# Save the model
xlm_roberta_model.save_pretrained('./fine-tuned-xlm-roberta-model')
xlm_roberta_tokenizer.save_pretrained('./fine-tuned-xlm-roberta-model')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,No log,0.100548
2,No log,0.056233
3,No log,0.044884


{'eval_loss': 0.0448840856552124, 'eval_runtime': 21.947, 'eval_samples_per_second': 9.751, 'eval_steps_per_second': 0.638, 'epoch': 3.0}


('./fine-tuned-xlm-roberta-model/tokenizer_config.json',
 './fine-tuned-xlm-roberta-model/special_tokens_map.json',
 './fine-tuned-xlm-roberta-model/sentencepiece.bpe.model',
 './fine-tuned-xlm-roberta-model/added_tokens.json',
 './fine-tuned-xlm-roberta-model/tokenizer.json')

In [9]:
from transformers import DistilBertTokenizerFast, DistilBertForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset
import torch

# Define the label mapping
label_mapping = {
    "O": 0,
    "I-LOC": 1,
    "I-PRICE": 2,
    "I-Product": 3,
    # Add other labels as needed
}

# Load the fast tokenizer and model for DistilBERT
distilbert_tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-multilingual-cased")
distilbert_model = DistilBertForTokenClassification.from_pretrained("distilbert-base-multilingual-cased", num_labels=len(label_mapping))

def tokenize_and_align_labels_distilbert(examples):
    tokenized_inputs = distilbert_tokenizer(
        examples['Token'].tolist(),
        padding=True,
        truncation=True,
        is_split_into_words=True,
    )

    labels = []
    word_ids = tokenized_inputs.word_ids(batch_index=0)  # Assuming batch_size is 1
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            label_ids.append(examples['Label'].values[word_idx])
        else:
            label_ids.append(-100)
        previous_word_idx = word_idx
    tokenized_inputs["labels"] = label_ids
    return tokenized_inputs

# Apply the function to each sentence
tokenized_datasets_distilbert = [tokenize_and_align_labels_distilbert(sentence) for _, sentence in df.groupby(df.index // 10)]

# Convert lists to PyTorch tensors
input_ids_distilbert = [torch.tensor(item['input_ids'], dtype=torch.long) for item in tokenized_datasets_distilbert]
attention_mask_distilbert = [torch.tensor(item['attention_mask'], dtype=torch.long) for item in tokenized_datasets_distilbert]
labels_distilbert = [torch.tensor(item['labels'], dtype=torch.long) for item in tokenized_datasets_distilbert]

# Pad sequences to ensure they have the same length
input_ids_distilbert = torch.nn.utils.rnn.pad_sequence(input_ids_distilbert, batch_first=True, padding_value=distilbert_tokenizer.pad_token_id)
attention_mask_distilbert = torch.nn.utils.rnn.pad_sequence(attention_mask_distilbert, batch_first=True, padding_value=0)
labels_distilbert = torch.nn.utils.rnn.pad_sequence(labels_distilbert, batch_first=True, padding_value=-100)

# Combine into a dataset
dataset_distilbert = Dataset.from_dict({
    'input_ids': input_ids_distilbert,
    'attention_mask': attention_mask_distilbert,
    'labels': labels_distilbert
})

# Set up training arguments for DistilBERT
training_args_distilbert = TrainingArguments(
    output_dir='./results_distilbert',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Fine-tune the model
trainer_distilbert = Trainer(
    model=distilbert_model,
    args=training_args_distilbert,
    train_dataset=dataset_distilbert,
    eval_dataset=dataset_distilbert,
)

trainer_distilbert.train()

# Evaluate the model
results_distilbert = trainer_distilbert.evaluate()
print(results_distilbert)

# Save the model
distilbert_model.save_pretrained('./fine-tuned-distilbert-model')
distilbert_tokenizer.save_pretrained('./fine-tuned-distilbert-model')


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,0.101364
2,No log,0.074666
3,No log,0.071841


{'eval_loss': 0.07184068113565445, 'eval_runtime': 30.5869, 'eval_samples_per_second': 6.996, 'eval_steps_per_second': 0.458, 'epoch': 3.0}


('./fine-tuned-distilbert-model/tokenizer_config.json',
 './fine-tuned-distilbert-model/special_tokens_map.json',
 './fine-tuned-distilbert-model/vocab.txt',
 './fine-tuned-distilbert-model/added_tokens.json',
 './fine-tuned-distilbert-model/tokenizer.json')

In [10]:
from transformers import BertTokenizerFast, BertForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset
import torch

# Define the label mapping
label_mapping = {
    "O": 0,
    "I-LOC": 1,
    "I-PRICE": 2,
    "I-Product": 3,
    # Add other labels as needed
}

# Load the fast tokenizer and model for mBERT
mbert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")
mbert_model = BertForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=len(label_mapping))

def tokenize_and_align_labels_mbert(examples):
    tokenized_inputs = mbert_tokenizer(
        examples['Token'].tolist(),
        padding=True,
        truncation=True,
        is_split_into_words=True,
    )

    labels = []
    word_ids = tokenized_inputs.word_ids(batch_index=0)  # Assuming batch_size is 1
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            label_ids.append(examples['Label'].values[word_idx])
        else:
            label_ids.append(-100)
        previous_word_idx = word_idx
    tokenized_inputs["labels"] = label_ids
    return tokenized_inputs

# Apply the function to each sentence
tokenized_datasets_mbert = [tokenize_and_align_labels_mbert(sentence) for _, sentence in df.groupby(df.index // 10)]

# Convert lists to PyTorch tensors
input_ids_mbert = [torch.tensor(item['input_ids'], dtype=torch.long) for item in tokenized_datasets_mbert]
attention_mask_mbert = [torch.tensor(item['attention_mask'], dtype=torch.long) for item in tokenized_datasets_mbert]
labels_mbert = [torch.tensor(item['labels'], dtype=torch.long) for item in tokenized_datasets_mbert]

# Pad sequences to ensure they have the same length
input_ids_mbert = torch.nn.utils.rnn.pad_sequence(input_ids_mbert, batch_first=True, padding_value=mbert_tokenizer.pad_token_id)
attention_mask_mbert = torch.nn.utils.rnn.pad_sequence(attention_mask_mbert, batch_first=True, padding_value=0)
labels_mbert = torch.nn.utils.rnn.pad_sequence(labels_mbert, batch_first=True, padding_value=-100)

# Combine into a dataset
dataset_mbert = Dataset.from_dict({
    'input_ids': input_ids_mbert,
    'attention_mask': attention_mask_mbert,
    'labels': labels_mbert
})

# Set up training arguments for mBERT
training_args_mbert = TrainingArguments(
    output_dir='./results_mbert',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Fine-tune the model
trainer_mbert = Trainer(
    model=mbert_model,
    args=training_args_mbert,
    train_dataset=dataset_mbert,
    eval_dataset=dataset_mbert,
)

trainer_mbert.train()

# Evaluate the model
results_mbert = trainer_mbert.evaluate()
print(results_mbert)

# Save the model
mbert_model.save_pretrained('./fine-tuned-mbert-model')
mbert_tokenizer.save_pretrained('./fine-tuned-mbert-model')


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,0.073177
2,No log,0.061392
3,No log,0.057371


{'eval_loss': 0.05737127363681793, 'eval_runtime': 60.5775, 'eval_samples_per_second': 3.533, 'eval_steps_per_second': 0.231, 'epoch': 3.0}


('./fine-tuned-mbert-model/tokenizer_config.json',
 './fine-tuned-mbert-model/special_tokens_map.json',
 './fine-tuned-mbert-model/vocab.txt',
 './fine-tuned-mbert-model/added_tokens.json',
 './fine-tuned-mbert-model/tokenizer.json')

In [11]:
pip install evaluate


In [12]:
import numpy as np
from evaluate import load
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Define a function to compute metrics
def compute_metrics(eval_pred):
    labels = eval_pred.label_ids
    preds = np.argmax(eval_pred.predictions, axis=2)

    # Remove ignored index (special tokens)
    true_labels = [
        [label for label in label_seq if label != -100]
        for label_seq in labels
    ]
    true_preds = [
        [pred for (pred, label) in zip(pred_seq, label_seq) if label != -100]
        for pred_seq, label_seq in zip(preds, labels)
    ]

    # Flatten the lists
    true_labels_flat = [item for sublist in true_labels for item in sublist]
    true_preds_flat = [item for sublist in true_preds for item in sublist]

    precision = precision_score(true_labels_flat, true_preds_flat, average='weighted')
    recall = recall_score(true_labels_flat, true_preds_flat, average='weighted')
    f1 = f1_score(true_labels_flat, true_preds_flat, average='weighted')
    accuracy = accuracy_score(true_labels_flat, true_preds_flat)

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy,
    }

# Evaluate each model
for model_name, trainer in [('XLM-Roberta', trainer_xlm_roberta), ('DistilBERT', trainer_distilbert), ('mBERT', trainer_mbert)]:
    eval_results = trainer.evaluate()
    print(f"\n{model_name} Evaluation Results:")
    print(eval_results)

    predictions, label_ids, metrics = trainer.predict(trainer.eval_dataset)
    metrics = compute_metrics(eval_pred=trainer.predict(trainer.eval_dataset))
    print(f"{model_name} Metrics: {metrics}")




XLM-Roberta Evaluation Results:
{'eval_loss': 0.0448840856552124, 'eval_runtime': 57.6722, 'eval_samples_per_second': 3.711, 'eval_steps_per_second': 0.243, 'epoch': 3.0}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


XLM-Roberta Metrics: {'precision': 0.9712388011588811, 'recall': 0.9855144855144855, 'f1': 0.9783245685132478, 'accuracy': 0.9855144855144855}



DistilBERT Evaluation Results:
{'eval_loss': 0.07184068113565445, 'eval_runtime': 30.9919, 'eval_samples_per_second': 6.905, 'eval_steps_per_second': 0.452, 'epoch': 3.0}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


DistilBERT Metrics: {'precision': 0.9712388011588811, 'recall': 0.9855144855144855, 'f1': 0.9783245685132478, 'accuracy': 0.9855144855144855}



mBERT Evaluation Results:
{'eval_loss': 0.05737127363681793, 'eval_runtime': 61.6684, 'eval_samples_per_second': 3.47, 'eval_steps_per_second': 0.227, 'epoch': 3.0}


mBERT Metrics: {'precision': 0.9712388011588811, 'recall': 0.9855144855144855, 'f1': 0.9783245685132478, 'accuracy': 0.9855144855144855}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
# Evaluate each model and store metrics
results_summary = {}

for model_name, trainer in [('XLM-Roberta', trainer_xlm_roberta), ('DistilBERT', trainer_distilbert), ('mBERT', trainer_mbert)]:
    eval_results = trainer.evaluate()
    metrics = compute_metrics(eval_pred=trainer.predict(trainer.eval_dataset))

    results_summary[model_name] = {
        'precision': metrics['precision'],
        'recall': metrics['recall'],
        'f1': metrics['f1'],
        'accuracy': metrics['accuracy'],
        'eval_loss': eval_results['eval_loss'],
        'eval_runtime': eval_results['eval_runtime'],
        'eval_samples_per_second': eval_results['eval_samples_per_second'],
        'eval_steps_per_second': eval_results['eval_steps_per_second']
    }

# Determine the best-performing model
best_model_name = max(results_summary, key=lambda k: results_summary[k]['f1'])  # Example criterion: highest F1 score

print(f"Best Performing Model: {best_model_name}")

# Print results summary for all models
for model_name, metrics in results_summary.items():
    print(f"\n{model_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

# Select the best model based on the best_model_name
best_model = None
if best_model_name == 'XLM-Roberta':
    best_model = trainer_xlm_roberta.model
elif best_model_name == 'DistilBERT':
    best_model = trainer_distilbert.model
elif best_model_name == 'mBERT':
    best_model = trainer_mbert.model


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best Performing Model: XLM-Roberta

XLM-Roberta Metrics:
precision: 0.9712388011588811
recall: 0.9855144855144855
f1: 0.9783245685132478
accuracy: 0.9855144855144855
eval_loss: 0.0448840856552124
eval_runtime: 21.8944
eval_samples_per_second: 9.774
eval_steps_per_second: 0.639

DistilBERT Metrics:
precision: 0.9712388011588811
recall: 0.9855144855144855
f1: 0.9783245685132478
accuracy: 0.9855144855144855
eval_loss: 0.07184068113565445
eval_runtime: 30.3068
eval_samples_per_second: 7.061
eval_steps_per_second: 0.462

mBERT Metrics:
precision: 0.9712388011588811
recall: 0.9855144855144855
f1: 0.9783245685132478
accuracy: 0.9855144855144855
eval_loss: 0.05737127363681793
eval_runtime: 60.4444
eval_samples_per_second: 3.54
eval_steps_per_second: 0.232


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
